In [220]:
from pandas import read_csv
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt


In [221]:
import pandas as pd
from data import citylearn_challenge_2022_phase_1 as competition_data
import os.path as osp

nsload_data_path = osp.join(osp.dirname(competition_data.__file__), "load_data.csv")
nsload_data = pd.read_csv(nsload_data_path)

# for i in range(5):
#     plt.plot(nsload_data["non_shiftable_load"][i*8760 : 8760 + i*8760])
#     plt.show()

In [222]:
selected_params = ["non_shiftable_load", "non_shiftable_load_future"] # "outdoor_dry_bulb", "outdoor_relative_humidity",
nsload_data = nsload_data[selected_params]

In [223]:
nsload_data

,non_shiftable_load,non_shiftable_load_future
0,0.851167,0.834600
1,0.834600,0.838167
2,0.838167,1.478433
3,1.478433,1.256200
4,1.256200,1.869517
...,...,...
43785,3.964600,2.730183
43786,2.730183,0.935833
43787,0.935833,2.155017
43788,2.155017,2.316400


In [224]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
nsload_data = nsload_data[0:8760]
df_load_scaled = sc.fit_transform(nsload_data)
print(df_load_scaled.shape)

sc2 = StandardScaler()
df_load_scaled_y = sc2.fit_transform(nsload_data[["non_shiftable_load_future"]])
print(df_load_scaled_y.shape)

(8760, 2)
(8760, 1)


In [225]:
hops = 24
no_records = df_load_scaled_y.shape[0]
no_cols = len(selected_params)
X_train = []
y_train = []
for i in range(hops, no_records):
    X_train.append(df_load_scaled[i-hops:i])
    y_train.append(df_load_scaled_y[i][0])
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train.shape)
print(y_train.shape)

(8736, 24, 2)
(8736,)


In [226]:
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

In [227]:
X_train_reshaped.shape

(8736, 24, 2)

In [228]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model=Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(hops,no_cols)))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train_reshaped, y_train, validation_split=0.2, epochs=50, batch_size=24, shuffle=False, workers=24, use_multiprocessing=True)

Epoch 1/50
292/292 [==============================] - 9s 24ms/step - loss: 0.8146 - val_loss: 0.9364
Epoch 2/50
292/292 [==============================] - 6s 21ms/step - loss: 0.6825 - val_loss: 0.8266
Epoch 3/50
292/292 [==============================] - 6s 21ms/step - loss: 0.6395 - val_loss: 0.8037
Epoch 4/50
292/292 [==============================] - 6s 21ms/step - loss: 0.6306 - val_loss: 0.7847
Epoch 5/50
292/292 [==============================] - 6s 21ms/step - loss: 0.6234 - val_loss: 0.7690
Epoch 6/50
292/292 [==============================] - 6s 21ms/step - loss: 0.6157 - val_loss: 0.7509
Epoch 7/50
292/292 [==============================] - 6s 21ms/step - loss: 0.6126 - val_loss: 0.7554
Epoch 8/50
292/292 [==============================] - 6s 21ms/step - loss: 0.6084 - val_loss: 0.7505
Epoch 9/50
292/292 [==============================] - 6s 21ms/step - loss: 0.6081 - val_loss: 0.7507
Epoch 10/50
292/292 [==============================] - 6s 22ms/step - loss: 0.6038 - val_lo

In [60]:
# units = dimensionality of the output space


def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape,
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

demo_model = create_RNN(hidden_units=2, dense_units=1, input_shape=(3,1), activation=['relu', 'relu'])